In [0]:
import pyspark.sql.functions as F

In [0]:
fato_envios_final = spark.table("grao.silver.fato_envios")
dim_localizacao = spark.table("grao.silver.dim_localizacao")
dim_metodo_envio = spark.table("grao.silver.dim_metodo_envio")
df_calendario = spark.table("grao.bronze.calendario") # Usando a do bronze, que já é uma dimensão


In [0]:
print("--- Criando Tabela Gold 1: Gold_Vendas_por_Destino ---")

gold_vendas_destino = (
    fato_envios_final
    .join(dim_localizacao, "id_localizacao", "left")
    .groupBy("Destino")
    .agg(
        F.sum("preco").alias("receita_total"),
        F.count("id_envio").alias("total_de_envios"),
        F.sum("qtd_itens").alias("total_itens_vendidos"),
        F.round(F.avg("preco"), 2).alias("valor_medio_por_envio")
    )
    .orderBy(F.col("receita_total").desc())
)


gold_vendas_destino.write.mode("overwrite").format("delta").saveAsTable("grao.gold.vendas_por_destino")
print("Tabela Gold 'vendas_por_destino' criada com sucesso.")